In [1]:
import pandas as pd
import numpy as np
import time
import random
import lightgbm as lgb
import time

import pickle
import matplotlib.pyplot as plt

In [2]:
from process import gen_second_data

In [3]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'

In [4]:
# original input
df_structures = pd.read_csv(FOLDER+'structures.csv')
df_test = pd.read_csv(FOLDER+'test.csv')

In [5]:
df_structures_idx = df_structures.set_index('molecule_name')
df_test_idx = df_test.set_index('molecule_name')

In [6]:
# input files
pred_train = '20190721_predict_train.pickle'
pred_val = '20190721_predict_val.pickle'
first_submission_name = '20190721_submission.csv'

map_train_pickle = '20190727_map_train.pickle'
map_val_pickle = '20190727_map_val.pickle'
map_test_pickle = '20190727_map_test.pickle'

dist_train_pickle = '20190727_dist_train.pickle'
dist_val_pickle = '20190727_dist_val.pickle'
dist_test_pickle = '20190727_dist_test.pickle'

In [7]:
# output files
temp_no = 0



In [8]:
# fixed slice for mols_train, mols_val
df_train = pd.read_pickle(OUTPUT + pred_train)[['id', 'molecule_name', 'atom_index_0','atom_index_1','type', 'scalar_coupling_constant','predict']]
df_val = pd.read_pickle(OUTPUT + pred_val)[['id', 'molecule_name', 'atom_index_0','atom_index_1','type', 'scalar_coupling_constant','predict']]

mols_train = df_train['molecule_name'].unique()
mols_val = df_val['molecule_name'].unique()

df = pd.concat([df_train, df_val], axis=0).sort_values('id')
df_idx = df.set_index('molecule_name')

In [ ]:
df_map_train = pd.read_pickle(OUTPUT + map_train_pickle)
df_map_val = pd.read_pickle(OUTPUT + map_val_pickle)
df_map_test = pd.read_pickle(OUTPUT + map_test_pickle)

df_dist_train = pd.read_pickle(OUTPUT + dist_train_pickle)
df_dist_val = pd.read_pickle(OUTPUT + dist_val_pickle)
df_dist_test = pd.read_pickle(OUTPUT + dist_test_pickle)

In [ ]:
df_map_train_idx = df_map_train.set_index('molecule_name')
df_map_val_idx = df_map_val.set_index('molecule_name')
df_map_test_idx = df_map_test.set_index('molecule_name')

df_dist_train_idx = df_dist_train.set_index('molecule_name')
df_dist_val_idx = df_dist_val.set_index('molecule_name')
df_dist_test_idx = df_dist_test.set_index('molecule_name')

In [ ]:
df_sc_dist_train = pd.DataFrame()
m = mols_train[0]
df_sc_dist_train_temp = gen_second_data(df_idx, df_map_train_idx, m, target_col='predict')
df_sc_dsit_train = pd.concat([df_sc_dist_train, df_sc_dist_train_temp], axis=0)
df_sc_dsit_train

In [ ]:
df_sc_dist_val = pd.DataFrame()
m = mols_val[0]
df_sc_dist_val_temp = gen_second_data(df_idx, df_map_val_idx, m, target_col='predict')
df_sc_dsit_val = pd.concat([df_sc_dist_val, df_sc_dist_val_temp], axis=0)
df_sc_dsit_val

In [ ]:
div = len(mols_train) // 8

df_train_map = pd.DataFrame()
df_train_dist = pd.DataFrame()

start = time.time()

# for m in mols_train[:10]:
for m in mols_train[div*temp_no:div*(temp_no+1)]:
    df_train_map_temp, df_train_dist_temp = gen_map_dist(df_idx, df_structures_idx, m)
    df_train_map = pd.concat([df_train_map, df_train_map_temp], axis=0)
    df_train_dist = pd.concat([df_train_dist, df_train_dist_temp], axis=0)
    
# df_train_map.to_pickle(OUTPUT + temp_map_train_pickle)
# df_train_dist.to_pickle(OUTPUT + temp_dist_train_pickle)

elapsed_time = time.time() - start
print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [ ]:
div = len(mols_val) // 8

df_val_map = pd.DataFrame()
df_val_dist = pd.DataFrame()

start = time.time()

# for m in mols_val[:10]:
for m in mols_val[div*temp_no:div*(temp_no+1)]:
    df_val_map_temp, df_val_dist_temp = gen_map_dist(df_idx, df_structures_idx, m)
    df_val_map = pd.concat([df_val_map, df_val_map_temp], axis=0)
    df_val_dist = pd.concat([df_val_dist, df_val_dist_temp], axis=0)
    
# df_val_map.to_pickle(OUTPUT + temp_map_val_pickle)
# df_val_dist.to_pickle(OUTPUT + temp_dist_val_pickle)

elapsed_time = time.time() - start
print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [ ]:
mols_test = df_test['molecule_name'].unique()

In [ ]:
div = len(mols_test) // 8

df_test_map = pd.DataFrame()
df_test_dist = pd.DataFrame()

start = time.time()

# for m in mols_test[:10]:
for m in mols_test[div*temp_no:div*(temp_no+1)]:
    df_test_map_temp, df_test_dist_temp = gen_map_dist(df_test_idx, df_structures_idx, m)
    df_test_map = pd.concat([df_test_map, df_test_map_temp], axis=0)
    df_test_dist = pd.concat([df_test_dist, df_test_dist_temp], axis=0)
    
# df_test_map.to_pickle(OUTPUT + temp_map_test_pickle)
# df_test_dist.to_pickle(OUTPUT + temp_dist_test_pickle)

elapsed_time = time.time() - start
print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [ ]:
df_test_map

In [ ]:
# df_map_train_list = []
# df_map_val_list = []
# df_map_test_list = []

# df_dist_train_list = []
# df_dist_val_list = []
# df_dist_test_list = []

# for i in range(8):
#     df_map_train_list.append(pd.read_pickle(OUTPUT + temp_map_train_pickle.format(i)))
#     df_map_val_list.append(pd.read_pickle(OUTPUT + temp_map_val_pickle.format(i)))
#     df_map_test_list.append(pd.read_pickle(OUTPUT + temp_map_test_pickle.format(i)))
    
#     df_dist_train_list.append(pd.read_pickle(OUTPUT + temp_dist_train_pickle.format(i)))
#     df_dist_val_list.append(pd.read_pickle(OUTPUT + temp_dist_val_pickle.format(i)))
#     df_dist_test_list.append(pd.read_pickle(OUTPUT + temp_dist_test_pickle.format(i)))
    
# df_map_train = pd.concat(df_map_train_list, axis=0)
# df_map_val = pd.concat(df_map_val_list, axis=0)
# df_map_test = pd.concat(df_map_test_list, axis=0)

# df_dist_train = pd.concat(df_dist_train_list, axis=0)
# df_dist_val = pd.concat(df_dist_val_list, axis=0)
# df_dist_test = pd.concat(df_dist_test_list, axis=0)

In [ ]:
# df_map_train.head()

In [ ]:
# df_map_train.to_pickle(OUTPUT + map_train_pickle)
# df_map_val.to_pickle(OUTPUT + map_val_pickle)
# df_map_test.to_pickle(OUTPUT + map_test_pickle)

# df_dist_train.to_pickle(OUTPUT + dist_train_pickle)
# df_dist_val.to_pickle(OUTPUT + dist_val_pickle)
# df_dist_test.to_pickle(OUTPUT + dist_test_pickle)

In [ ]:
# print(df_train.shape,
# df_val.shape,
# df_test.shape)

In [ ]:
# print(df_map_train.shape, 
# df_map_val.shape,
# df_map_test.shape,
# df_dist_train.shpape,
# df_dist_val.shape,
# df_dist_test.shape)